<a href="https://colab.research.google.com/github/SwaksharDeb/Activity-recognition/blob/master/HAR_paper1_v2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dependecies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import array
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from numpy import savetxt
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
import scipy.stats as stats
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from math import sqrt
import math

In [ ]:
from scipy.fftpack import fft
from scipy.stats import spearmanr
from scipy.stats import pearsonr

# Import dataset

In [ ]:
"""Import Datasets"""
dataset_user04 = pd.read_csv("final_field_user04_dataset.csv")
dataset_user07 = pd.read_csv("final_field_user07_dataset.csv")
dataset_user08 = pd.read_csv("final_field_user08_dataset.csv")
dataset_user18 = pd.read_csv("final_field_user18_dataset.csv")
dataset_user38 = pd.read_csv("final_field_user38_dataset.csv")
dataset_user51 = pd.read_csv("final_field_user51_dataset.csv")
dataset_lab = pd.read_csv("lab data localized.csv")

dataset = pd.concat([dataset_lab, dataset_user04, dataset_user07, dataset_user08, dataset_user18, dataset_user38, dataset_user51], ignore_index = True)


# Data processing

In [ ]:
""" Data processing"""
X = dataset.iloc[:, [0, 1, 2]].values
Y = dataset.iloc[:, 3].values
Y = Y - 1

In [ ]:
X.shape,Y.shape

((1867777, 3), (1867777,))

## low pass filter

In [ ]:
low_pass_filter = np.zeros((X.shape[0], X.shape[1]))
for i in range(len(X)):
    for j in range(X.shape[1]):
        if i == 0:
            low_pass_filter[i][j] = X[i][j]
        else:
            low_pass_filter[i][j] = 0.8*X[i-1][j] + (1-0.8)*X[i][j]

## Frame preparation

In [ ]:
# frame preparation
Fs = 4
frame_size = Fs*20 # 80 # 20 seconds of data
hop_size =  Fs*10

def get_frames(df, frame_size, hop_size,label_data):

    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, len(df) - frame_size, hop_size):
        value = df[i: i + frame_size, :]
        #y = df[i: i + frame_size, 1]
        #z = df[i: i + frame_size, 2]

        # Retrieve the most often used label in this segment
        label = stats.mode(label_data[i: i + frame_size])[0][0]
        frames.append([value])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

row_frame, row_label = get_frames(X, frame_size, hop_size, Y)
low_pass_frame, low_pass_label = get_frames(low_pass_filter, frame_size, hop_size, Y)

In [ ]:
row_frame.shape, low_pass_frame.shape

((46693, 80, 3), (46693, 80, 3))

## Feature calculation

In [ ]:
#feature calculation
mean = np.mean(low_pass_frame, axis = 1)  # mean along (x,y,z) axis
std = np.std(low_pass_frame, axis = 1)  # standard deviation along (x,y,z) axis

rms_x = []  # root mean square along x axis
value = 0
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][0] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_x.append(value)
    value = 0

rms_y = []  # root mean square along y axis
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][1] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_y.append(value)
    value = 0

rms_z = []  # root mean square z axis
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][2] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_z.append(value)
    value = 0

In [ ]:
lengths = []  # length of the accelerometer vector
length = 0
nx, ny, nz = row_frame.shape[0], row_frame.shape[1], row_frame.shape[2]
acc_vector = np.reshape(row_frame,(nx, ny*nz))

for i in range(acc_vector.shape[0]):
    count = 0
    for j in range(acc_vector.shape[1]):
        length += acc_vector[i][j]**2
        count += 1

        if count == 3:
            length = sqrt(length)
            lengths.append(length)
            length = 0
            count = 0

avc = []  # avc feature

polling_rate = 4 #Hz

for i in range(0, len(lengths), frame_size):
    value = 0
    iterable = lengths[i:i+frame_size]
    for j in range(len(iterable) - 1, 1, -1):
        value += abs(iterable[j] - iterable[j - 1])

    value = value / (frame_size / polling_rate)
    avc.append(value)


In [ ]:
max_min = []

for i in range(0, len(lengths), frame_size):
    iterable = lengths[i:i+frame_size]

    max_min.append(max(iterable) - min(iterable))


In [ ]:
angle = np.zeros((low_pass_frame.shape[0],low_pass_frame.shape[1],low_pass_frame.shape[2]))  # inclenation angle
angles = []
length = 0
for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        length = low_pass_frame[i,j,0]**2 + low_pass_frame[i,j,1]**2 + low_pass_frame[i,j,2]**2
        length = sqrt(length)
        if length == 0:
            print("length is zero in index row= ",i,"column= ",j)
        else:
            angle_x = math.acos(low_pass_frame[i,j,0]/length)
            angle_y = math.acos(low_pass_frame[i,j,1]/length)
            angle_z = math.acos(low_pass_frame[i,j,2]/length)
            angle[i][j][0] = angle_x
            angle[i][j][1] = angle_y
            angle[i][j][2] = angle_z
nx, ny, nz = angle.shape
angle = np.reshape(angle,(nx,ny*nz))


In [ ]:
fft_x = []  # fast fourier transform cofficient along x axis
for i in range(row_frame.shape[0]):
  fft_x.append(fft(row_frame[i,:,0]))

fft_y = []  # fast fourier transform cofficient along x axis
for i in range(row_frame.shape[0]):
  fft_y.append(fft(row_frame[i,:,1]))

fft_z = []  # fast fourier transform cofficient along x axis
for i in range(row_frame.shape[0]):
  fft_z.append(fft(row_frame[i,:,2]))

energy_x = []
modulus_square = 0
for i in fft_x:
  for j in range(len(fft_x[0])):
    modulus_square += i[j].real**2 + i[j].imag**2
  energy_x.append(modulus_square/len(fft_x[0]))
  modulus_square = 0

energy_y = []
for i in fft_y:
  for j in range(len(fft_y[0])):
    modulus_square += i[j].real**2 + i[j].imag**2
  energy_y.append(modulus_square/len(fft_x[0]))
  modulus_square = 0

energy_z = []
for i in fft_z:
  for j in range(len(fft_z[0])):
    modulus_square += i[j].real**2 + i[j].imag**2
  energy_z.append(modulus_square/len(fft_x[0]))
  modulus_square = 0

In [ ]:
input_vector = np.zeros((row_frame.shape[0], angle.shape[1] + 14))  # input vector for the algorithm
for i in range(input_vector.shape[0]):
    input_vector[i][0] = mean[i][0]
    input_vector[i][1] = mean[i][1]
    input_vector[i][2] = mean[i][2]
    input_vector[i][3] = std[i][0]
    input_vector[i][4] = std[i][1]
    input_vector[i][5] = std[i][2]
    input_vector[i][6] = rms_x[i]
    input_vector[i][7] = rms_y[i]
    input_vector[i][8] = rms_z[i]
    input_vector[i][9] = max_min[i]
    input_vector[i][10] = avc[i]
    input_vector[i,11] = energy_x[i]
    input_vector[i,12] = energy_y[i]
    input_vector[i,13] = energy_z[i]
    #input_vector[i,9] = correlation[i,0]
    #input_vector[i,10] = correlation[i,1]
    #input_vector[i,11] = correlation[i,2]


for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        input_vector[i][j+14] = angle[i][j]

In [ ]:
input_vector.shape, row_label.shape

((46693, 254), (46693,))

# Standarization

In [ ]:
sc = StandardScaler()
input_vector = sc.fit_transform(input_vector)

# Trian Test split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(input_vector, row_label, test_size = 0.2, random_state = 0, stratify = row_label)

In [ ]:
X_train.shape, y_train.shape

((37354, 254), (37354,))

# Algorithm

## Neural Network

In [ ]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(X_train.shape[1], )))
#model.add(Dropout(0.2))
model.add(Dense(units=64, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=128, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(units=324, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=12, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs = 50, batch_size =64, validation_data= (X_val, y_val), verbose=1)

Epoch 1/50
584/584 [==============================] - 2s 3ms/step - loss: 1.0648 - accuracy: 0.6022 - val_loss: 1.0056 - val_accuracy: 0.6159
Epoch 2/50
584/584 [==============================] - 2s 3ms/step - loss: 0.9760 - accuracy: 0.6211 - val_loss: 0.9625 - val_accuracy: 0.6318
Epoch 3/50
584/584 [==============================] - 2s 3ms/step - loss: 0.9410 - accuracy: 0.6318 - val_loss: 0.9553 - val_accuracy: 0.6127
Epoch 4/50
584/584 [==============================] - 2s 3ms/step - loss: 0.9230 - accuracy: 0.6345 - val_loss: 0.9205 - val_accuracy: 0.6390
Epoch 5/50
584/584 [==============================] - 1s 2ms/step - loss: 0.9021 - accuracy: 0.6409 - val_loss: 0.9037 - val_accuracy: 0.6413
Epoch 6/50
584/584 [==============================] - 1s 2ms/step - loss: 0.8868 - accuracy: 0.6468 - val_loss: 0.9018 - val_accuracy: 0.6340
Epoch 7/50
584/584 [==============================] - 1s 3ms/step - loss: 0.8749 - accuracy: 0.6503 - val_loss: 0.8989 - val_accuracy: 0.6348
Epoch 

In [ ]:
y_pred = model.predict(X_val)

In [ ]:
y_pred_ = np.zeros((y_pred.shape[0]))
for i in range(y_pred.shape[0]):
    y_pred_[i] = np.argmax(y_pred[i,:])

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred_, y_val)

In [ ]:
pd.DataFrame(cm)

,0,1,2,3,4,5,6,7,8,9,10,11
0,17,1,4,12,19,2,0,1,0,1,1,0
1,2,7,4,2,6,3,4,1,2,1,2,4
2,6,2,98,44,77,3,20,5,2,2,1,12
3,39,12,87,979,436,50,148,9,14,11,13,55
4,62,12,310,946,4769,34,220,8,19,15,7,89
5,2,3,2,8,9,6,7,2,1,0,3,0
6,8,11,17,75,68,6,171,2,1,1,2,6
7,0,1,1,1,1,3,1,0,1,1,1,0
8,1,0,0,2,3,0,1,0,1,3,1,0
9,1,2,1,0,2,0,2,0,0,5,1,5


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred_)

In [ ]:
per_class_accuracy = {}
for i in range(cm.shape[0]):
    accuracy = cm[i,i]/sum(cm[i,:])
    per_class_accuracy[i] = accuracy

In [ ]:
per_class_accuracy

{0: 0.29310344827586204,
 1: 0.18421052631578946,
 2: 0.3602941176470588,
 3: 0.5283324338909876,
 4: 0.7347095979047913,
 5: 0.13953488372093023,
 6: 0.46467391304347827,
 7: 0.0,
 8: 0.08333333333333333,
 9: 0.2631578947368421,
 10: 0.07692307692307693,
 11: 0.6554054054054054}

# Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
#y_pred = classifier.predict(X_val)

## Adaboosting

Our base model is random forest. 

If $\textit{class_weight = 'balanced'}$ does not work well, then try the munual class weight that I define below. just type $\textit{class_weight = class_weights}$

In [ ]:
class_weights = {}
for classes in np.unique(y_train):
    class_weights[classes] = len(X_train)/(np.count_nonzero(y_train == classes)
                                     *len(np.unique(y_train))

class 4,5,11 weight are manually set to 1 since if we lower their weight, their accuracy decrease. So, if we give them just normal weight overall accuracy may increase

In [ ]:
class_weights[4] = 1  
class_weights[3] = 1
class_weights[11] = 1

Try different $\textit{max_depth}$ (10,15), increase the $\textit{n_estimator}$ of $\textit{adaboost}$ 

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
estimator = RandomForestClassifier(n_estimators = 100, max_depth = 5, class_weight = 'balanced', criterion = 'entropy', random_state = 0)
classifier_ada = AdaBoostClassifier(base_estimator = estimator, n_estimators=50, random_state=0)
classifier_ada.fit(X_train, y_train)

In [ ]:
y_pred = classifier_ada.predict(X_val)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)

In [ ]:
pd.DataFrame(cm)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_train, y_pred)

In [ ]:
per_class_accuracy = {}
for i in range(cm.shape[0]):
    accuracy = cm[i,i]/sum(cm[i,:])
    per_class_accuracy[i] = accuracy

# Ensemble model

In [ ]:
y_pred_nn = model.predict(X_train)
y_pred_ = np.zeros((y_pred_nn.shape[0]))
for i in range(y_pred_nn.shape[0]):
    y_pred_[i] = np.argmax(y_pred_nn[i,:])
y_pred_nn = y_pred_

y_pred_rf = classifier.predict(X_train)

#X_train_ensemble = np.concatenate((y_pred_nn, y_pred_rf),axis=1)
X_train_ensemble = np.vstack((y_pred_nn, y_pred_rf)).T

from sklearn.svm import SVC
blinder = SVC(kernel = 'linear', random_state = 1)
blinder.fit(X_train_ensemble, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
y_pred = classifier.predict(X_val)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)

In [ ]:
pd.DataFrame(cm)

,0,1,2,3,4,5,6,7,8,9,10,11
0,11,0,11,27,89,0,1,0,0,0,0,0
1,1,3,3,17,24,1,2,0,0,0,0,0
2,1,0,99,73,352,0,1,0,0,0,0,1
3,1,3,18,1233,825,2,8,0,0,0,0,0
4,1,0,44,595,4764,0,5,0,0,1,0,0
5,0,0,3,65,33,5,2,3,0,0,0,0
6,5,0,5,173,270,3,123,0,0,0,0,0
7,1,0,2,13,13,0,1,1,0,0,0,0
8,0,0,2,16,24,0,1,0,2,1,0,0
9,0,1,3,13,23,0,1,0,0,1,1,0


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred)

In [ ]:
accuracy

0.678338151836385